In [1]:
import os
import pandas as pd
import numpy as np


import time
from datetime import datetime
import random

import pickle

import matplotlib.pyplot as plt
import os, random, torch
import numpy as np

In [2]:
def setSeeds(seed = 42):
    # 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)    
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
setSeeds(seed = 42)

In [4]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 
train_path = '../input/data/train_dataset/train_data.csv'
test_path = '../input/data/train_dataset/test_data.csv'

df = pd.read_csv(train_path)
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
test_df = pd.read_csv(test_path)
total_df = pd.concat([df, test_df[test_df.answerCode!=-1]], ignore_index=True)

In [5]:
def convert_time(s):
    timestamp = time.mktime(datetime.strptime(s, '%Y-%m-%d %H:%M:%S').timetuple())
    return int(timestamp)

In [6]:
def create_elapsed_time(df):
    prev_timestamp = df.groupby(['userID','testId','day'])[['time_stamp']].shift()
    df['elapsed_time'] = df['time_stamp'] - prev_timestamp['time_stamp']
    df['elapsed_time'] = df['elapsed_time'].fillna(0)
    return df

In [7]:
def create_lag_time(df):
    start_end_id_by_user_test = df.groupby(['userID','testId','day']).apply(lambda x: (x.index.values[0], x.index.values[-1])).reset_index()
    start_end_id_by_user_test = start_end_id_by_user_test.sort_values(by=[0]).reset_index(drop=True)
    start_row_id_by_user = start_end_id_by_user_test.groupby('userID').apply(lambda x: x.index.values[0])
    
    lag_time_list = [0]*len(df)
    for start_row, end_row in start_end_id_by_user_test [0][1:]:
        start_time = df.time_stamp[start_row]
        prev_time = df.time_stamp[start_row-1]
        lag_time = start_time - prev_time
        lag_time_list[start_row:end_row+1] = [lag_time]*(end_row-start_row+1)
    
    # 사용자가 바뀌는 부분 첫 시험지 lag_time은 0으로 변경
    for user_start_idx in start_row_id_by_user:
        start, end = start_end_id_by_user_test .loc[user_start_idx][0]
        lag_time_list[start:end+1] = [0]*(end-start+1)
        
    df['lag_time'] = lag_time_list
    
    return df

In [8]:
def feature_engineering(df, total_df):
    df['time_stamp'] = df['Timestamp'].apply(convert_time)
    df['day'] = df.Timestamp.apply(lambda x:x.split()[0])
    df = create_elapsed_time(df)  # elapsed_time
    df = create_lag_time(df)  # lag_time
    
    # -- 과거 feature
    # 이전까지 맞은 문제 수 (피처 계산에만 사용)
    df['prior_acc_count'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1)).fillna(0)
    # 이전까지 푼 문제 수  (피처 계산에만 사용)
    df['prior_quest_count'] = df.groupby('userID')['answerCode'].cumcount().fillna(0)
    # 이전 문제까지의 정답률
    df['prior_acc'] = (df['prior_acc_count']/df['prior_quest_count']).fillna(0)

    return df

In [9]:
df = feature_engineering(df, total_df)

In [10]:
# 로그 스케일링
# 로그 스케일 적용할 column
log1p_cols = ['elapsed_time',
              'lag_time'
             ]

df[log1p_cols] = np.log1p(df[log1p_cols])

In [11]:
# 범주형, 수치형 feature
cate_cols = ['assessmentItemID', 'testId', 'KnowledgeTag']

cont_cols = ['elapsed_time',
             'lag_time',
             'prior_acc'
            ]

In [12]:
import argparse

In [13]:
def parse_args(mode='train'):
    parser = argparse.ArgumentParser()

    
    parser.add_argument('--seed', default=42, type=int, help='seed')
    
    parser.add_argument('--device', default='cpu', type=str, help='cpu or gpu')

    parser.add_argument('--data_dir', default='/opt/ml/input/data/train_dataset', type=str, help='data directory')
    parser.add_argument('--asset_dir', default='asset/', type=str, help='data directory')
    
    parser.add_argument('--file_name', default='train_data.csv', type=str, help='train file name')
    
    parser.add_argument('--model_dir', default='models/', type=str, help='model directory')
    parser.add_argument('--model_name', default='model.pt', type=str, help='model file name')

    parser.add_argument('--output_dir', default='output/', type=str, help='output directory')
    parser.add_argument('--test_file_name', default='test_data.csv', type=str, help='test file name')
    
    parser.add_argument('--max_seq_len', default=20, type=int, help='max sequence length')
    parser.add_argument('--num_workers', default=4, type=int, help='number of workers')

    # 모델
    parser.add_argument('--hidden_dim', default=64, type=int, help='hidden dimension size')
    parser.add_argument('--n_layers', default=2, type=int, help='number of layers')
    parser.add_argument('--n_heads', default=2, type=int, help='number of heads')
    parser.add_argument('--drop_out', default=0.2, type=float, help='drop out rate')
    
    # 훈련
    parser.add_argument('--n_epochs', default=100, type=int, help='number of epochs')
    parser.add_argument('--batch_size', default=64, type=int, help='batch size')
    parser.add_argument('--lr', default=0.0001, type=float, help='learning rate')
    parser.add_argument('--clip_grad', default=10, type=int, help='clip grad')
    parser.add_argument('--patience', default=5, type=int, help='for early stopping')
    

    parser.add_argument('--log_steps', default=50, type=int, help='print log per n steps')
    

    ### 중요 ###
    parser.add_argument('--model', default='lstm', type=str, help='model type')
    parser.add_argument('--optimizer', default='adam', type=str, help='optimizer type')
    parser.add_argument('--scheduler', default='plateau', type=str, help='scheduler type')
    
    args = parser.parse_args([])

    return args

In [14]:
args = parse_args(mode='train')
device = "cuda" if torch.cuda.is_available() else "cpu"
args.device = device
print(device)

cuda


In [15]:
from dkt.dataloader import Preprocess

In [16]:
preprocess = Preprocess(args)

In [17]:
# 객체명._클래스명__private함수명으로 접근 가능
df = preprocess._Preprocess__preprocessing(df, is_train=True)

In [18]:
# 추후 feature를 embedding할 시에 embedding_layer의 input 크기를 결정할때 사용
args.n_questions = len(np.load(os.path.join(args.asset_dir,'assessmentItemID_classes.npy')))
args.n_test = len(np.load(os.path.join(args.asset_dir,'testId_classes.npy')))
args.n_tag = len(np.load(os.path.join(args.asset_dir,'KnowledgeTag_classes.npy')))

In [19]:
df = df.sort_values(by=['userID','Timestamp'], axis=0)

In [20]:
# ['assessmentItemID', 'testId', 'KnowledgeTag'], ['elapsed_time','lag_time','prior_acc']
columns = ['userID','answerCode'] + cate_cols + cont_cols

In [21]:
group = df[columns].groupby('userID').apply(
                lambda r: tuple([r[col].values for col in columns[1:]])
            )

In [22]:
tot_data = group.values

In [24]:
train_data, valid_data = preprocess.split_data(tot_data)

In [25]:
def get_loaders(args, train, valid):

    pin_memory = True  # False
    train_loader, valid_loader = None, None
    
    if train is not None:
        trainset = CustomDKTDataset(train, args)
        train_loader = torch.utils.data.DataLoader(trainset, num_workers=args.num_workers, shuffle=True,
                            batch_size=args.batch_size, pin_memory=pin_memory, collate_fn=collate)
    if valid is not None:
        valset = CustomDKTDataset(valid, args)
        valid_loader = torch.utils.data.DataLoader(valset, num_workers=args.num_workers, shuffle=False,
                            batch_size=args.batch_size, pin_memory=pin_memory, collate_fn=collate)

    return train_loader, valid_loader

In [26]:
class CustomDKTDataset(torch.utils.data.Dataset):
    def __init__(self, data, args):
        self.data = data
        self.args = args

    def __getitem__(self, index):
        row = self.data[index]

        # 각 data의 sequence length
        seq_len = len(row[0])
        
        # 'assessmentItemID', 'testId', 'KnowledgeTag'
        correct, question, test, tag = row[0], row[1], row[2], row[3]  
        cate_cols = [test, question, tag, correct]

        # 'elapsed_time','lag_time','prior_acc'
        cont_cols = list(row[4:])

        # max seq len을 고려하여서 이보다 길면 자르고 아닐 경우 그대로 냅둔다
        if seq_len > args.max_seq_len:
            for i, col in enumerate(cate_cols):
                cate_cols[i] = col[-args.max_seq_len:]

            for i, col in enumerate(cont_cols):
                cont_cols[i] = col[-args.max_seq_len:]
            mask = np.ones(args.max_seq_len, dtype=np.int16)
        else:
            # 원래 sequence가 있는 길이부분만 1
            mask = np.zeros(args.max_seq_len, dtype=np.int16)
            mask[-seq_len:] = 1

        # mask도 columns 목록에 포함시킴
        cate_cols.append(mask)

        # np.array -> torch.tensor 형변환
        for i, col in enumerate(cate_cols):
            cate_cols[i] = torch.tensor(col)

        for i, col in enumerate(cont_cols):
            cont_cols[i] = torch.tensor(col)

        return cont_cols+cate_cols
    

    def __len__(self):
        return len(self.data)

In [27]:
def collate(batch):
    col_n = len(batch[0])  
    col_list = [[] for _ in range(col_n)]
    # input으로 들어오는 batch는 DKTDataset의 output으로  
    # 기존 baseline에서 맨 마지막에 위치하는 tensor는 mask
    # mask는 DKTDataset에서 생성될 때 args.max_seq_len에 맞춰서 생성됨
    # DKTDataset의 output을 cont+cate로 수정하면 그대로 mask가 맨 마지막에 위치함
    max_seq_len = len(batch[0][-1]) 

    # batch의 값들을 각 column끼리 그룹화
    for row in batch:  # row: feature values
        for i, col in enumerate(row):  # col: feature value
            pre_padded = torch.zeros(max_seq_len)
            pre_padded[-len(col):] = col  # 앞부분이 0으로 padding됨
            col_list[i].append(pre_padded)


    for i, _ in enumerate(col_list):
        col_list[i] =torch.stack(col_list[i])
    
    return tuple(col_list)


In [31]:

# 배치 전처리
def process_batch(batch, args):

    test, question, tag, correct, mask = batch[-5:]
    cont = batch[:-5] 
    
    # change to float
    mask = mask.type(torch.FloatTensor)
    correct = correct.type(torch.FloatTensor)

    # interaction: 과거 정답 여부를 다음 시퀀스에 추가적인 feature로 사용하게끔 한칸 시프트 해준 feature
    #  interaction을 임시적으로 correct를 한칸 우측으로 이동한 것으로 사용
    #    saint의 경우 decoder에 들어가는 input이다
    interaction = correct + 1 # 패딩을 위해 correct값에 1을 더해준다. (정답 2, 오답 1)
    interaction = interaction.roll(shifts=1, dims=1)
    interaction_mask = mask.roll(shifts=1, dims=1)
    interaction_mask[:, 0] = 0
    interaction = (interaction * interaction_mask).to(torch.int64)  # 가장 마지막으로 푼 문제를 제외하고 정답 2, 오답 1
    # print(interaction)
    # exit()
    #  test_id, question_id, tag
    test = ((test + 1) * mask).to(torch.int64)
    question = ((question + 1) * mask).to(torch.int64)
    tag = ((tag + 1) * mask).to(torch.int64)
    
    # cont features도 padding을 위해 1을 더함
    for i in range(len(cont)):
        cont[i] = ((cont[i]+1)*mask).to(torch.float32)


    # device memory로 이동
    test = test.to(args.device)
    question = question.to(args.device)
    tag = tag.to(args.device)
    correct = correct.to(args.device)
    mask = mask.to(args.device)
    interaction = interaction.to(args.device)
    for i in range(len(cont)):
        cont[i] = cont[i].to(args.device)

    return (test, question, tag, correct, mask, interaction, cont)


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import copy
import math

try:
    from transformers.modeling_bert import BertConfig, BertEncoder, BertModel    
except:
    from transformers.models.bert.modeling_bert import BertConfig, BertEncoder, BertModel    



In [97]:
class LSTM(nn.Module):

    def __init__(self, args, cont_col_count=3):
        super(LSTM, self).__init__()
        self.args = args
        self.device = args.device
    
        self.cont_col_count = cont_col_count
        
        self.hidden_dim = self.args.hidden_dim
        self.n_layers = self.args.n_layers
        

        # Embedding 
        # interaction은 현재 correct로 구성되어있다. correct(1, 2) + padding(0)
        self.embedding_interaction = nn.Embedding(3, self.hidden_dim//3)
        self.embedding_test = nn.Embedding(self.args.n_test + 1, self.hidden_dim//3)
        self.embedding_question = nn.Embedding(self.args.n_questions + 1, self.hidden_dim//3)
        self.embedding_tag = nn.Embedding(self.args.n_tag + 1, self.hidden_dim//3)
        
        # embedding cate projection
        #self.cate_proj = nn.Linear((self.hidden_dim//3)*4, self.hidden_dim)
        self.cate_proj = nn.Sequential(
                            nn.Linear((self.hidden_dim//3)*4, self.hidden_dim),
                            nn.LayerNorm(self.hidden_dim)
                            )  
        
        # cont embedding
        self.cont_bn = nn.BatchNorm1d(self.cont_col_count)
        self.cont_proj =  nn.Sequential(
                nn.Linear(self.cont_col_count, self.hidden_dim),
                nn.LayerNorm(self.hidden_dim)
                )
        
        # embedding combination projection
        self.comb_proj = nn.Linear(self.hidden_dim*2, self.hidden_dim)

        self.lstm = nn.LSTM(self.hidden_dim,
                            self.hidden_dim,
                            self.n_layers,
                            batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(self.hidden_dim, 1)

        self.activation = nn.Sigmoid()

    def init_hidden(self, batch_size):
        h = torch.zeros(
            self.n_layers,
            batch_size,
            self.hidden_dim)
        h = h.to(self.device)

        c = torch.zeros(
            self.n_layers,
            batch_size,
            self.hidden_dim)
        c = c.to(self.device)

        return (h, c)

    def forward(self, input):

        test, question, tag, _, mask, interaction, cont = input

        batch_size = interaction.size(0)

        # Embedding

        embed_interaction = self.embedding_interaction(interaction)
        embed_test = self.embedding_test(test)
        embed_question = self.embedding_question(question)
        embed_tag = self.embedding_tag(tag)
        
        cate_embed = torch.cat([embed_interaction,
                                embed_test,
                                embed_question,
                                embed_tag,], 2
                                )
        cate_embed = self.cate_proj(cate_embed)
        

        cont_x = torch.cat(cont, 1)
        cont_bn_x = self.cont_bn(cont_x.view(-1, self.cont_col_count))
        # view 할 때 맨 앞을 -1로 해야함,
        # 맨 마지막 batch는 실제 batch보다 작아서 고정된 값으로 놓으면 에러남
        cont_bn_x = cont_bn_x.view(-1, args.max_seq_len, self.cont_col_count)
        cont_embed = self.cont_proj(cont_bn_x)
        
        comb_embed = torch.cat([cate_embed, cont_embed], 2)
        X = self.comb_proj(comb_embed)

        hidden = self.init_hidden(batch_size)
        out, hidden = self.lstm(X, hidden)
        out = out.contiguous().view(batch_size, -1, self.hidden_dim)  # .contiguous(): 새로운 텐서를 반환

        out = self.fc(out)
        preds = self.activation(out).view(batch_size, -1)

        return preds

In [98]:

def train(train_loader, model, optimizer, args):
    model.train()

    total_preds = []
    total_targets = []
    losses = []
    for step, batch in enumerate(train_loader):
        input = process_batch(batch, args)
        preds = model(input)
        targets = input[3] # correct


        loss = compute_loss(preds, targets)
        update_params(loss, model, optimizer, args)


        if step % args.log_steps == 0:
            print(f"Training steps: {step} Loss: {str(loss.item())}")
        
        # predictions
        preds = preds[:,-1]
        targets = targets[:,-1]

        if args.device == 'cuda':
            preds = preds.to('cpu').detach().numpy()
            targets = targets.to('cpu').detach().numpy()
        else: # cpu
            preds = preds.detach().numpy()
            targets = targets.detach().numpy()
        
        total_preds.append(preds)
        total_targets.append(targets)
        losses.append(loss)
      

    total_preds = np.concatenate(total_preds)
    total_targets = np.concatenate(total_targets)

    # Train AUC / ACC
    auc, acc = get_metric(total_targets, total_preds)
    loss_avg = sum(losses)/len(losses)
    print(f'TRAIN AUC : {auc} ACC : {acc}')
    return auc, acc, loss_avg
    

def validate(valid_loader, model, args):
    model.eval()

    total_preds = []
    total_targets = []
    for step, batch in enumerate(valid_loader):
        input = process_batch(batch, args)

        preds = model(input)
        targets = input[3] # correct


        # predictions
        preds = preds[:,-1]
        targets = targets[:,-1]
    
        if args.device == 'cuda':
            preds = preds.to('cpu').detach().numpy()
            targets = targets.to('cpu').detach().numpy()
        else: # cpu
            preds = preds.detach().numpy()
            targets = targets.detach().numpy()

        total_preds.append(preds)
        total_targets.append(targets)

    total_preds = np.concatenate(total_preds)
    total_targets = np.concatenate(total_targets)

    # Train AUC / ACC
    auc, acc = get_metric(total_targets, total_preds)
    
    print(f'VALID AUC : {auc} ACC : {acc}\n')

    return auc, acc, total_preds, total_targets



def inference(args, test_data):
    
    model = load_model(args)
    model.eval()
    _, test_loader = get_loaders(args, None, test_data)
    
    
    total_preds = []
    
    for step, batch in enumerate(test_loader):
        input = process_batch(batch, args)

        preds = model(input)
        

        # predictions
        preds = preds[:,-1]
        

        if args.device == 'cuda':
            preds = preds.to('cpu').detach().numpy()
        else: # cpu
            preds = preds.detach().numpy()
            
        total_preds+=list(preds)

    write_path = os.path.join(args.output_dir, "output.csv")
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)    
    with open(write_path, 'w', encoding='utf8') as w:
        print("writing prediction : {}".format(write_path))
        w.write("id,prediction\n")
        for id, p in enumerate(total_preds):
            w.write('{},{}\n'.format(id,p))



In [99]:
import os
import torch
import numpy as np

#from dkt.dataloader import get_loaders
from dkt.optimizer import get_optimizer
from dkt.scheduler import get_scheduler
from dkt.criterion import get_criterion
from dkt.metric import get_metric
from dkt.trainer import get_lr, compute_loss, update_params, save_checkpoint, load_model

import wandb
import time
import datetime
import gc


In [100]:
train_loader, valid_loader = get_loaders(args, train_data, valid_data)

# only when using warmup scheduler
args.total_steps = int(len(train_loader.dataset) / args.batch_size) * (args.n_epochs)
args.warmup_steps = args.total_steps // 10

model = LSTM(args)
model.to(args.device)

optimizer = get_optimizer(model, args)
scheduler = get_scheduler(optimizer, args)

In [101]:
print(len(train_loader))
print(len(valid_loader))

74
32


In [102]:
wandb.init(project='dkt', config=vars(args), tags=[args.model], name=f'cont_cate_{args.model}_{args.n_epochs}epochs')    

wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [103]:
best_auc = -1
early_stopping_counter = 0
for epoch in range(args.n_epochs):

    print(f"Start Training: Epoch {epoch + 1}")
    start = time.time()
    ### TRAIN
    train_auc, train_acc, train_loss = train(train_loader, model, optimizer, args)

    ### VALID
    auc, acc,_ , _ = validate(valid_loader, model, args)

    sec = time.time() - start
    times = str(datetime.timedelta(seconds=sec)).split(".")
    times = times[0]
    print(f'<<<<<<<<<<  {epoch + 1} EPOCH spent : {times}  >>>>>>>>>>')

    ### TODO: model save or early stopping
    wandb.log({"epoch": epoch, "train_loss": train_loss, "train_auc": train_auc, "train_acc":train_acc,
              "valid_auc":auc, "valid_acc":acc, "Learning_rate": get_lr(optimizer),})
    if auc > best_auc:
        best_auc = auc
        # torch.nn.DataParallel로 감싸진 경우 원래의 model을 가져옵니다.
        model_to_save = model.module if hasattr(model, 'module') else model
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model_to_save.state_dict(),
            },
            args.model_dir, 'model.pt',
        )
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= args.patience:
            print(f'EarlyStopping counter: {early_stopping_counter} out of {args.patience}')
            break

    # scheduler
    if args.scheduler == 'plateau':
        scheduler.step(best_auc)
    else:
        scheduler.step()

Start Training: Epoch 1
Training steps: 0 Loss: 0.7000912427902222
Training steps: 50 Loss: 0.6918453574180603
TRAIN AUC : 0.5685841187994971 ACC : 0.5420221843003413
VALID AUC : 0.7147598687244672 ACC : 0.6462686567164179

<<<<<<<<<<  1 EPOCH spent : 0:00:02  >>>>>>>>>>
saving model ...
Start Training: Epoch 2
Training steps: 0 Loss: 0.6881948113441467
Training steps: 50 Loss: 0.6821510791778564
TRAIN AUC : 0.7183864265612308 ACC : 0.6452645051194539
VALID AUC : 0.7352134020853728 ACC : 0.6810945273631841

<<<<<<<<<<  2 EPOCH spent : 0:00:02  >>>>>>>>>>
saving model ...
Start Training: Epoch 3
Training steps: 0 Loss: 0.6826542615890503
Training steps: 50 Loss: 0.6533534526824951
TRAIN AUC : 0.7325656596229215 ACC : 0.681740614334471
VALID AUC : 0.7364095209474532 ACC : 0.6761194029850747

<<<<<<<<<<  3 EPOCH spent : 0:00:02  >>>>>>>>>>
saving model ...
Start Training: Epoch 4
Training steps: 0 Loss: 0.6134753823280334
Training steps: 50 Loss: 0.6390873193740845
TRAIN AUC : 0.738791346